In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [27]:
class BPR(nn.Module):
    def __init__(self):
        super(BPR, self).__init__()
        
        self.W = None             # user matrix
        self.H = None             # item matrix
        
        self.Wsc = None        # scorer
        self.Hsc = None 
        
        self.user_items = {}
        self.dev_user_items = {}
        
        self.uid = None
        self.iid = None
        
        self.uid_dict = None      # serialize uid and iid
        self.iid_dict = None      #  {(original id in dataset): (serial_idx)}
        self.uid_dict_rev = None  # reverse key and value
        self.iid_dict_rev = None  #  {(serial_idx): (original id in dataset)}
        
        self.rating_exp = None   # softmax sum
        self.rating_exp_mul_H = None
        
    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio, shuffle = True, random_state = 5)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test
    
    def split(self, df, train_size=0.8, test_size=0.1):
        self.uid = np.asarray(list(set(df.iloc[:,0].values)))
        self.iid = np.asarray(list(set(df.iloc[:,1].values)))
        self.uid.sort()
        self.iid.sort()
        self.uid_dict = dict(zip(self.uid, [i for i in range(len(self.uid))]))
        self.iid_dict = dict(zip(self.iid, [i for i in range(len(self.iid))]))
        self.uid_dict_rev = {v: k for k, v in self.uid_dict.items()}
        self.iid_dict_rev = {v: k for k, v in self.iid_dict.items()}
        
        train, test = self._split(df, train_size)
        test, dev = self._split(test, test_size / (1 - train_size))
        return train, test, dev
    
    def generate_train_batch(self, batch, sets):
        train = []
        for b in range(batch):
            u = self.uid[np.random.randint(0, len(self.uid))]
            i = sets[u][np.random.randint(0, len(sets[u]))]
            j = self.iid[np.random.randint(0, len(self.iid))]
            while j in sets[u]:
                j = self.iid[np.random.randint(0, len(self.iid))]
            train.append([self.uid_dict[u], self.iid_dict[i], self.iid_dict[j]])
        return np.asarray(train) 
    
#     def forward2(self, uids, iids, device):
#         prob = []
        
#         self.rating_exp = torch.zeros(len(self.uid)).to(device)
#         self.rating_exp_mul_H = torch.zeros([len(self.uid), self.H.shape[1]]).to(device)
        
#         for x in range(uids.size):
#             uid = uids[x]
#             iid = iids[x]
            
#             if self.rating_exp[uid] == 0:
#                 ori_idxs = self.user_items[self.uid_dict_rev[uid] ]
#                 emb_idxs = [self.iid_dict[ori_idx] for ori_idx in ori_idxs]
#                 user_emb = self.Wsc[uid]
#                 item_emb = self.Hsc[emb_idxs]
                
#                 user_item_exp_sc = torch.exp(torch.mv(item_emb, user_emb))
               
#                 self.rating_exp[uid] = torch.sum(user_item_exp_sc)
#                 self.rating_exp_mul_H[uid] = torch.sum(user_item_exp_sc.unsqueeze(1).repeat(1, self.H.shape[1]) * item_emb, dim = 0)        
                
#             prob.append(torch.exp(torch.dot(self.Wsc[uid], self.Hsc[iid])) / self.rating_exp[uid]) 
#         print(self.rating_exp_mul_H[uids])
#         return torch.tensor(prob).to(device)

    def forward(self, uids, iids, device):
        self.rating_exp = torch.zeros(len(self.uid)).to(device)
        self.rating_exp_mul_H = torch.zeros([len(self.uid), self.H.shape[1]]).to(device)
        
        # 处理 idx 得到 embedded Wu Hi
        ori_uids = [self.uid_dict_rev[uid] for uid in uids]
        emb_idxs = [[ self.iid_dict[ori_idx] for ori_idx in self.user_items[ori_uid] ] for ori_uid in ori_uids]
        item_emb = nn.utils.rnn.pad_sequence([self.Hsc[emb_idx] for emb_idx in emb_idxs], batch_first=True)
        user_emb = self.Wsc[uids][:, None, :]
        
        # 计算批次内 user_item 得分
        user_item_exp_sc = torch.sum(item_emb * user_emb, dim = -1)
        mask = (user_item_exp_sc != 0).type(torch.float32)
        # 取指数， mask 保证补 0 位还是 0
        user_item_exp_sc = torch.exp(user_item_exp_sc) * mask
        
        # 计算指数和
        self.rating_exp_mul_H[uids] = torch.sum(user_item_exp_sc.unsqueeze(2).repeat(1, 1, self.H.shape[1]) * item_emb, dim = 1)
        self.rating_exp[uids] = torch.sum(user_item_exp_sc, dim = 1)
        #返回 softmax probablilty of item i among user_items
        return torch.exp(torch.sum(self.Wsc[uids] * self.Hsc[iids], dim = 1)) / self.rating_exp[uids]
            
    def fit_dds(self, df, dev, k, stepsize=0.05, max_iter=10, batch=10000):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        # 初始化 W，H
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)   
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01) 
        
        # 初始化 scorer
        self.Wsc = torch.rand(len(self.uid), k).to(device) * 0.01    
        self.Hsc = torch.rand(len(self.iid), k).to(device) * 0.01  
        
        # 创建字典：用户u对应他访问过的所有items集合    
        self.user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_dict()
        self.dev_user_items = dev.groupby(dev.columns[0])[dev.columns[1]].apply(lambda x: np.array(x)).to_dict()
        
        # 主模型优化器        
        optimizer = optim.Adam([self.W, self.H], lr = stepsize)
        
        for x in range(max_iter):            
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.user_items)
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            # 评分器概率分布，forward 返回 softmax 概率分布
            score_prob = self.forward(u, i, device)     
            
            # 主模型参数更新
            optimizer.zero_grad() 
            score_loss = -torch.mean(score_prob * torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb-j_emb),dim = 1))))
            bpr_loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            score_loss.backward()
            optimizer.step()

            # 训练集上 W,H 的梯度
            W_grad_sum = self.W.grad.clone()
            H_grad_sum = self.H.grad.clone()
            
            # 对数概率分布下 Wsc, Hsc 梯度
            log_prob_Wsc_grad = torch.zeros((len(self.uid), k)).to(device)
            log_prob_Hsc_grad = torch.zeros((len(self.iid), k)).to(device)
            log_prob_Wsc_grad[u] = self.Hsc[i] - self.rating_exp_mul_H[u] / self.rating_exp[u].unsqueeze(1).repeat(1, k)
            log_prob_Hsc_grad[i] = self.Wsc[u] * ((1 - score_prob).unsqueeze(1).repeat(1, k))
            
            #取 dev uij三元组
            uij = self.generate_train_batch(5000, self.dev_user_items)
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            # 计算 dev 集上 W,H 的梯度
            optimizer.zero_grad()
            dev_loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            dev_loss.backward()
            W_grad_dev_sum = self.W.grad.clone()    
            H_grad_dev_sum = self.H.grad.clone()

            # 计算 reward: reward 为 W,H 在训练集和 dev 集上的梯度积    
            r_W = torch.sum(W_grad_sum * W_grad_dev_sum, dim=1)
            r_H = torch.sum(H_grad_sum * H_grad_dev_sum, dim=1)
            r_W = r_W.unsqueeze(1).repeat(1, k)
            r_H = r_H.unsqueeze(1).repeat(1, k)

            # Wsc，Hsc 更新
            self.Wsc += r_W * log_prob_Wsc_grad
            self.Hsc += r_H * log_prob_Hsc_grad

            if ( x + 1 ) % 10 == 0:
                print(f"Iteration: {x+1}, BPR loss: {bpr_loss.item()}")
    
    def fit_ori(self, df, k, stepsize=0.05, max_iter=10, batch=10000):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)    # 初始化 W，H
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01)  
        
        # 创建字典：用户u对应他访问过的所有items集合
        self.user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_dict()
        
        optimizer = optim.Adam([self.W, self.H], lr = stepsize)     # 主模型优化器
        for x in range(max_iter):
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.user_items)
            
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            optimizer.zero_grad()
            loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            loss.backward()
            optimizer.step()
            
            if ( x + 1 ) % 10 == 0:
                print(f"Iteration: {x+1}, BPR loss: {loss.item()}")
    
    def _predict(self, uid, items, n):
        item = [self.iid_dict[i] for i in items]
        user = self.uid_dict[uid]
        scores = torch.mv(self.H[item], self.W[user])
        top_N = list(zip(items, scores.detach().cpu().numpy()))
        return sorted(top_N, key=lambda s: s[1], reverse=True)[:n]

    def NDCG(self, uid, test, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
        test_user = test[test.iloc[:, 0] == uid]
        rating = self._predict(uid, test_user.iloc[:, 1].values, n)
        irating =sorted(test_user.iloc[:, 2].values, reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating): n = len(irating)  
        for i in range(n):
            r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
            dcg += 1.0 * (2**r - 1) / math.log(i + 2, 2)
            idcg += 1.0 * (2**irating[i] - 1) / math.log(i + 2, 2)
        return dcg / idcg

    def performance(self, test, n):      # Output recall@n, precision@n, NDCG@n
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        for i in self.uid:
            # Items that User i hasn't tried in training set
            unknown_items = np.setdiff1d(self.iid, self.user_items[i])
            # Items that User i actually tried in testing set
            known_items = test[test.iloc[:, 0] == i].iloc[:, 1].values
            
            #目标：预测 unknown items 中的top_N，若击中test中的items，则为有效预测
            ru = self._predict(i, unknown_items, n)
            for item, pui in ru:
                if item in known_items:
                    hit += 1
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, test, n)

        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [28]:
df1 = pd.read_csv("./ml-100k/u.data", sep="\t", names=['user id', 'item id', 'rating', 'timestamp'])
df2 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

### 100K

In [24]:
model1 = BPR()
model2 = BPR()
train1, test1, dev1 = model1.split(df1)
train2, test2, dev2 = model2.split(df1)
print(train1.shape)
print(test1.shape)
print(dev1.shape)

(79619, 4)
(9942, 4)
(10439, 4)


### 100K Pure BPR

In [5]:
%%time
model1.fit_ori(train1, k = 50, max_iter = 500)

Iteration: 10, BPR loss: 0.34321820735931396
Iteration: 20, BPR loss: 0.2845911681652069
Iteration: 30, BPR loss: 0.1951943188905716
Iteration: 40, BPR loss: 0.1442578136920929
Iteration: 50, BPR loss: 0.12120641767978668
Iteration: 60, BPR loss: 0.10743703693151474
Iteration: 70, BPR loss: 0.10088685899972916
Iteration: 80, BPR loss: 0.09578598290681839
Iteration: 90, BPR loss: 0.09169719368219376
Iteration: 100, BPR loss: 0.08609083294868469
Iteration: 110, BPR loss: 0.078936368227005
Iteration: 120, BPR loss: 0.0768565833568573
Iteration: 130, BPR loss: 0.07956204563379288
Iteration: 140, BPR loss: 0.07245401293039322
Iteration: 150, BPR loss: 0.08067290484905243
Iteration: 160, BPR loss: 0.08181027323007584
Iteration: 170, BPR loss: 0.07737747579813004
Iteration: 180, BPR loss: 0.07716871052980423
Iteration: 190, BPR loss: 0.07087761908769608
Iteration: 200, BPR loss: 0.0784105733036995
Iteration: 210, BPR loss: 0.07919120043516159
Iteration: 220, BPR loss: 0.07167080044746399
Iter

In [6]:
%%time
n = 10
rec, pre, ndcg = model1.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.10498409331919406
Recall@10: 0.0995775497887749
NDCG@10: 0.8212617984439188
CPU times: total: 1min 24s
Wall time: 2min 10s


### 100K BPR + Data Selection

In [11]:
%%time
model2.fit_dds(train1, dev1, k = 50, max_iter = 500)

Iteration: 10, BPR loss: 0.36946091055870056
Iteration: 20, BPR loss: 0.3186231553554535
Iteration: 30, BPR loss: 0.24832765758037567
Iteration: 40, BPR loss: 0.16737757623195648
Iteration: 50, BPR loss: 0.14268428087234497
Iteration: 60, BPR loss: 0.12022192031145096
Iteration: 70, BPR loss: 0.11052170395851135
Iteration: 80, BPR loss: 0.11082051694393158
Iteration: 90, BPR loss: 0.10729774832725525
Iteration: 100, BPR loss: 0.09982667863368988
Iteration: 110, BPR loss: 0.09177907556295395
Iteration: 120, BPR loss: 0.08649598062038422
Iteration: 130, BPR loss: 0.09156856685876846
Iteration: 140, BPR loss: 0.08470410108566284
Iteration: 150, BPR loss: 0.08146990835666656
Iteration: 160, BPR loss: 0.08098555356264114
Iteration: 170, BPR loss: 0.08361070603132248
Iteration: 180, BPR loss: 0.07724657654762268
Iteration: 190, BPR loss: 0.08311904966831207
Iteration: 200, BPR loss: 0.08271296322345734
Iteration: 210, BPR loss: 0.08003785461187363
Iteration: 220, BPR loss: 0.0781513601541519

In [12]:
%%time
n = 10
rec, pre, ndcg = model2.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.10869565217391304
Recall@10: 0.10309796821565077
NDCG@10: 0.8186659970475956
CPU times: total: 1min
Wall time: 2min 16s


### 1M

In [34]:
model3 = BPR()
model4 = BPR()
train3, test3, dev3 = model3.split(df2)
train4, test4, dev4 = model4.split(df2)
print(train3.shape)
print(test3.shape)
print(dev3.shape)

(797758, 4)
(99692, 4)
(102759, 4)


### 1M Pure BPR

In [35]:
model3.fit_ori(train3, k = 20, max_iter = 500)

Iteration: 10, BPR loss: 0.37315675616264343
Iteration: 20, BPR loss: 0.35101908445358276
Iteration: 30, BPR loss: 0.3575577139854431
Iteration: 40, BPR loss: 0.33635327219963074
Iteration: 50, BPR loss: 0.32147300243377686
Iteration: 60, BPR loss: 0.31918105483055115
Iteration: 70, BPR loss: 0.2890665531158447
Iteration: 80, BPR loss: 0.2785409688949585
Iteration: 90, BPR loss: 0.26115262508392334
Iteration: 100, BPR loss: 0.24021996557712555
Iteration: 110, BPR loss: 0.23213715851306915
Iteration: 120, BPR loss: 0.20239047706127167
Iteration: 130, BPR loss: 0.21163949370384216
Iteration: 140, BPR loss: 0.20129649341106415
Iteration: 150, BPR loss: 0.21305468678474426
Iteration: 160, BPR loss: 0.20476847887039185
Iteration: 170, BPR loss: 0.19826120138168335
Iteration: 180, BPR loss: 0.19827216863632202
Iteration: 190, BPR loss: 0.18471236526966095
Iteration: 200, BPR loss: 0.2061823606491089
Iteration: 210, BPR loss: 0.19735468924045563
Iteration: 220, BPR loss: 0.1969861090183258
It

In [36]:
n = 10
rec, pre, ndcg = model3.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.10902317880794701
Recall@10: 0.06605344460939694
NDCG@10: 0.7998781089801381


### 1M BPR + Data Selection

In [37]:
model4.fit_dds(train3, dev3, k = 20, max_iter = 500)

Iteration: 10, BPR loss: 0.4070197641849518
Iteration: 20, BPR loss: 0.3906244933605194
Iteration: 30, BPR loss: 0.3765500485897064
Iteration: 40, BPR loss: 0.3531898856163025
Iteration: 50, BPR loss: 0.3345780670642853
Iteration: 60, BPR loss: 0.31240397691726685
Iteration: 70, BPR loss: 0.3207390010356903
Iteration: 80, BPR loss: 0.2927694320678711
Iteration: 90, BPR loss: 0.2807278335094452
Iteration: 100, BPR loss: 0.27128997445106506
Iteration: 110, BPR loss: 0.2651827931404114
Iteration: 120, BPR loss: 0.2417038530111313
Iteration: 130, BPR loss: 0.22670380771160126
Iteration: 140, BPR loss: 0.21507641673088074
Iteration: 150, BPR loss: 0.22000204026699066
Iteration: 160, BPR loss: 0.19937869906425476
Iteration: 170, BPR loss: 0.21532931923866272
Iteration: 180, BPR loss: 0.21381667256355286
Iteration: 190, BPR loss: 0.20928165316581726
Iteration: 200, BPR loss: 0.21017666161060333
Iteration: 210, BPR loss: 0.21056024730205536
Iteration: 220, BPR loss: 0.20587335526943207
Iterati

In [38]:
n = 10
rec, pre, ndcg = model4.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.1022682119205298
Recall@10: 0.06196083938530674
NDCG@10: 0.8012535429823792
